# ProfessorAI
Ask questions and get answers based on lecture content.

In [47]:
import ipywidgets as widgets
import langchain
import pinecone
import openai
import tiktoken
import os

In [86]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.chains import ConversationalRetrievalChain

from langchain.memory import ConversationBufferMemory

from langchain.vectorstores import Pinecone
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

In [49]:
# IMPORTANT Only run this if you want to set your keys using environment variables
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
PINCONE_API_ENV = os.environ.get("PINECONE_API_ENV")

In [53]:

embeddings_engine = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [51]:
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINCONE_API_ENV
)
index_name = "diskstrukt2023"

In [54]:
#db = Pinecone.from_existing_index(index_name=index_name, embedding=embeddings_engine)
db = Chroma(persist_directory="../transcription_db", embedding_function=embeddings_engine)

Select what AI model to use to get your answer here:

In [88]:
# Valid model names as of making are
    #"gpt-4": 8192,
    #"gpt-4-0314": 8192,
    #"gpt-4-0613": 8192,
    #"gpt-4-32k": 32768,
    #"gpt-4-32k-0314": 32768,
    #"gpt-4-32k-0613": 32768,
    #"gpt-3.5-turbo": 4096,
    #"gpt-3.5-turbo-0301": 4096,
    #"gpt-3.5-turbo-0613": 4096,
    #"gpt-3.5-turbo-16k": 16385,
    #"gpt-3.5-turbo-16k-0613": 16385,
    #"text-ada-001": 2049,
    #"ada": 2049,
    #"text-babbage-001": 2040,
    #"babbage": 2049,
    #"text-curie-001": 2049,
    #"curie": 2049,
    #"davinci": 2049,
    #"text-davinci-003": 4097,
    #"text-davinci-002": 4097,
    #"code-davinci-002": 8001,
    #"code-davinci-001": 8001,
    #"code-cushman-002": 2048,
    #"code-cushman-001": 2048,
model = "gpt-3.5-turbo"

#llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
llm = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY, model=model)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=db.as_retriever())
conversational_qa_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=db.as_retriever(), memory=memory)

Set your questions here

(Questions should work in any language just fine)

In [56]:
question = "Was sind binominalkoeffizienten"

In [70]:
qa_chain.run(query=question, verbose=True)

'\n\nBinomische Koeffizienten sind mathematische Ausdrücke, die die Anzahl der möglichen Kombinationen von Elementen in einer Gruppe angeben. Sie werden häufig verwendet, um die Wahrscheinlichkeit bestimmter Ereignisse zu berechnen.'

In [77]:
conversational_qa_chain({"question": "Wie berechnet man diese?"})


{'question': 'Wie berechnet man diese?',
 'chat_history': [HumanMessage(content='Was sind binominalkoeffizienten', additional_kwargs={}, example=False),
  AIMessage(content='\n\nBinomische Koeffizienten sind mathematische Ausdrücke, die die Anzahl der möglichen Kombinationen von Elementen in einer Gruppe angeben. Sie werden häufig verwendet, um die Wahrscheinlichkeit bestimmter Ereignisse zu berechnen.', additional_kwargs={}, example=False),
  HumanMessage(content='Wie berechnet man diese?', additional_kwargs={}, example=False),
  AIMessage(content='\n\nDie binomischen Koeffizienten können mit der binomischen Formel berechnet werden. Diese Formel lautet: (n über k) = n! / (k! (n-k)!). Hierbei steht n für die Anzahl der Elemente in der Grundmenge, und k steht für die Anzahl der Elemente, die ausgewählt werden.', additional_kwargs={}, example=False)],
 'answer': '\n\nDie binomischen Koeffizienten können mit der binomischen Formel berechnet werden. Diese Formel lautet: (n über k) = n! / (

In [92]:
import gradio as gr

def answer_question(question):
    question_result = conversational_qa_chain({"question": question})
    return question_result["answer"]

QA_Interface = gr.Interface(fn=answer_question, 
    inputs="text", 
    outputs="text",
    layout="vertical"
    )

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])

    def respond(message, chat_history):
        bot_message = answer_question(message)
        chat_history.append((message, bot_message))
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch()



/tmp/ipykernel_32081/1497134276.py:7: GradioDeprecationWarning: `layout` parameter is deprecated, and it has no effect
  QA_Interface = gr.Interface(fn=answer_question,


Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.
